In [114]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification
from sklearn.model_selection import train_test_split
from sklearn import metrics
import itertools
import pandas as pd

In [115]:
ipos = pd.read_csv("scoop_ipos.csv")
# get labels
ipos["Underpriced"] = ipos["Underpriced"].astype(int)
ipo_labels = ipos["Underpriced"].tolist()
# get features
ipos = ipos.select_dtypes(['float64', 'float32', int])
ipo_features = ipos._get_numeric_data().columns.values.tolist()[1:-1]
print("Possible Features:", ipo_features)
# convert data types of all possible feature columns
for ipo_feature in ipo_features :
    ipos[ipo_feature] = ipos[ipo_feature].astype('float32').notnull()

Possible Features: ['Offer Price', 'Opening Price', '1st Day Close', '1st Day Percent Change', 'Change Opening', 'Change Close', 'Year']


In [145]:
# get columns for specified features
ipo_features = ipo_features[:-1]
ipo_features_data = ipos[['Offer Price', 'Opening Price']]
# split dataset to trianing set and test set
ipo_features_data_train, ipo_features_data_test, ipo_labels_train, ipo_labels_test = train_test_split(ipo_features_data, ipo_labels, test_size=0.3)
# create classifier 
clf = RandomForestClassifier(n_estimators=100)
# train the model
clf.fit(ipo_features_data_train, ipo_labels_train)
# predict
ipo_labels_pred = clf.predict(ipo_features_data_test)
# TODO save trained models

In [146]:
# check accuracy
print("Accuracy:", metrics.accuracy_score(ipo_labels_test, ipo_labels_pred))

Accuracy: 0.7936962750716332


In [147]:
# find feature importance
feature_imp = pd.Series(clf.feature_importances_,index=['Offer Price', 'Opening Price']).sort_values(ascending=False)
feature_imp

Offer Price      0.615385
Opening Price    0.384615
dtype: float64